In [1]:
from instructRNN.analysis.model_analysis import *
from instructRNN.tasks.tasks import *
from instructRNN.data_loaders.perfDataFrame import PerfDataFrame
from instructRNN.tasks.task_criteria import isCorrect
from instructRNN.instructions.instruct_utils import train_instruct_dict, test_instruct_dict
from instructRNN.tasks.task_factory import STIM_DIM
from instructRNN.instructions.instruct_utils import get_task_info
from instructRNN.trainers.decoder_trainer import *

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib import cm
from matplotlib.lines import Line2D
from matplotlib.ticker import MaxNLocator
import torch
from sklearn.preprocessing import normalize

from scipy.ndimage import gaussian_filter1d
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

from instructRNN.models.full_models import *
from instructRNN.plotting.plotting import *
from instructRNN.analysis.model_eval import *
from instructRNN.analysis.model_analysis import *
from instructRNN.tasks.tasks import DICH_DICT
import pickle

from sklearn.metrics.pairwise import cosine_similarity
from instructRNN.analysis.decoder_analysis import * 

In [3]:
simpleNet = SimpleNetPlus()
simpleNet.load_model('NN_simData/swap_holdouts/swap0/simpleNetPlus', suffix='_seed0')
rule_reps = get_rule_reps(simpleNet, use_rule_encoder=True)
#task_reps = get_task_reps(simpleNet, epoch=None, max_var=True)
rule_reps

array([[[0.        , 0.10294278, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 1.57921374, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.12451614, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.        , 0.        , 0.        , ..., 0.20414522,
         0.33085451, 0.        ]],

       [[0.        , 0.42365247, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.02634974, 0.        , ..., 0.        ,
         0.19766884, 0.        ]]])

In [ ]:
TASK_LIST[:12]

sns.heatmap(np.corrcoef(task_reps[:12, 0, :, 0]), yticklabels=TASK_LIST[:12])


In [108]:
def fit_rule_PCs(rule_encoding_set): 
    embedder = PCA()
    embedder.fit(rule_encoding_set.reshape(-1, rule_encoding_set.shape[-1]))
    return embedder

rule_reps = get_rule_reps(simpleNet, use_rule_encoder=True)
pcs, var_exp = reduce_rep(np.swapaxes(rule_reps, 0, 2), range(50))
pcs.shape

(64, 50)


(64, 1, 50)

In [ ]:
directions_dict = {}
for key, values in DICH_DICT.items():
    directions_dict[key] = np.array([rule_reps[TASK_LIST.index(dich_pair[1])]-rule_reps[TASK_LIST.index(dich_pair[0])] for dich_pair in values])


from sklearn.metrics.pairwise import cosine_similarity

dich_key = 'dich7'
sns.heatmap(cosine_similarity(directions_dict[dich_key].squeeze(), pcs.squeeze().T))
plt.show()


In [26]:
def get_unit_traj_corr(task_reps): 
    corr_mat = np.empty((len(TASK_LIST), len(TASK_LIST), task_reps.shape[-1]))
    task_mean_reps = task_reps.mean(1)
    for i in range(task_reps.shape[-1]): 
        corr_mat[...,i] = np.corrcoef(task_mean_reps[..., i])
    return corr_mat

task_reps = get_task_reps(simpleNet, epoch=None)
corr_mat  = get_unit_traj_corr(task_reps)    
sns.heatmap(corr_mat.mean(-1))
plt.show()

